Implementazione di https://medium.com/eni-digitalks/a-simple-recommender-system-using-pagerank-4a63071c8cbf

Da: https://grouplens.org/datasets/movielens/ scaricare [ml-latest-small.zip](https://files.grouplens.org/datasets/movielens/ml-latest-small.zip)

## recommended for education and development
 
### MovieLens Latest Datasets

These datasets will change over time, and are not appropriate for reporting research results. We will keep the download links stable for automated downloads. We will not archive or make available previously released versions.

_Small_: 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users. Last updated 9/2018.

    README.html
[ml-latest-small.zip](https://files.grouplens.org/datasets/movielens/ml-latest-small.zip) (size: 1 MB)

_Full_: approximately 33,000,000 ratings and 2,000,000 tag applications applied to 86,000 movies by 330,975 users. Includes tag genome data with 14 million relevance scores across 1,100 tags. Last updated 9/2018.

    README.html
[ml-latest.zip](https://files.grouplens.org/datasets/movielens/ml-latest.zip) (size: 335 MB) 

Permalink: https://grouplens.org/datasets/movielens/latest/

In [ ]:
#pip install pandas

In [19]:
import pandas as pd
import time

start_time = time.time()
#ratings = pd.read_csv("../ml-latest-small/ratings.csv")
ratings = pd.read_csv("../../movieBigDataset/ratings.csv")
print("--- %s seconds ---" % (time.time() - start_time))

--- 22.76940155029297 seconds ---


In [20]:
#movies = pd.read_csv("../ml-latest-small/movies.csv")
movies = pd.read_csv("../../movieBigDataset/movies.csv")

In [21]:
user_movie_matrix = pd.merge(ratings, movies, on="movieId", how="inner")

In [22]:
user_movie_matrix.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,1225734739,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,110,4.0,1225865086,Braveheart (1995),Action|Drama|War
2,1,158,4.0,1225733503,Casper (1995),Adventure|Children
3,1,260,4.5,1225735204,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
4,1,356,5.0,1225735119,Forrest Gump (1994),Comedy|Drama|Romance|War


In [5]:
print(user_movie_matrix["rating"].unique())

[4.  5.  3.  2.  1.  4.5 3.5 2.5 0.5 1.5]


In [23]:
mapping_score = {
    0.5:-1.2,
    1:-1.1,
    1.5:-1,
    2:-0.5,
    2.5:-0.1,
    3:0.1,
    3.5:0.5,
    4:1,
    4.5:1.1,
    5:1.2
}

Basandoci sulle ratings presenti nel dataset vado a proiettare il grafo movie_movie che mi consente di trovare la similarità tra i film. Il vettore di mappaggio è scelto arbitrariamente e serve a decidere quanto pesano le varie ratings.

In [13]:
#pip install networkx

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import networkx as nx

user_movie_graph = nx.Graph()

start_time = time.time()
for _, row in user_movie_matrix.iterrows():
    user_movie_graph.add_node(row["userId"], bipartite=0)
    user_movie_graph.add_node(row["title"], bipartite=1, genre=row["genres"], movieId=row["movieId"])
    user_movie_graph.add_edge(row["userId"], row["title"], weight=mapping_score[row["rating"]])
print("--- %s seconds ---" % (time.time() - start_time))

In [8]:
print(f"Nodes in the graph: {list(user_movie_graph.nodes(data=True))[:10]}")

Nodes in the graph: [(1, {'bipartite': 0}), ('Toy Story (1995)', {'bipartite': 1, 'genre': 'Adventure|Animation|Children|Comedy|Fantasy', 'movieId': 1}), ('Grumpier Old Men (1995)', {'bipartite': 1, 'genre': 'Comedy|Romance', 'movieId': 3}), ('Heat (1995)', {'bipartite': 1, 'genre': 'Action|Crime|Thriller', 'movieId': 6}), ('Seven (a.k.a. Se7en) (1995)', {'bipartite': 1, 'genre': 'Mystery|Thriller', 'movieId': 47}), ('Usual Suspects, The (1995)', {'bipartite': 1, 'genre': 'Crime|Mystery|Thriller', 'movieId': 50}), ('From Dusk Till Dawn (1996)', {'bipartite': 1, 'genre': 'Action|Comedy|Horror|Thriller', 'movieId': 70}), ('Bottle Rocket (1996)', {'bipartite': 1, 'genre': 'Adventure|Comedy|Crime|Romance', 'movieId': 101}), ('Braveheart (1995)', {'bipartite': 1, 'genre': 'Action|Drama|War', 'movieId': 110}), ('Rob Roy (1995)', {'bipartite': 1, 'genre': 'Action|Drama|Romance|War', 'movieId': 151})]


In [9]:
print(f"Edges in the graph: {list(user_movie_graph.edges(data=True))[:10]}")

Edges in the graph: [(1, 'Toy Story (1995)', {'weight': 1}), (1, 'Grumpier Old Men (1995)', {'weight': 1}), (1, 'Heat (1995)', {'weight': 1}), (1, 'Seven (a.k.a. Se7en) (1995)', {'weight': 1.2}), (1, 'Usual Suspects, The (1995)', {'weight': 1.2}), (1, 'From Dusk Till Dawn (1996)', {'weight': 0.1}), (1, 'Bottle Rocket (1996)', {'weight': 1.2}), (1, 'Braveheart (1995)', {'weight': 1}), (1, 'Rob Roy (1995)', {'weight': 1.2}), (1, 'Canadian Bacon (1995)', {'weight': 1.2})]


In [13]:
print(list(user_movie_graph.edges(data=True))[0][2]['weight'])

1


In [10]:
# Check memory consumption
import sys

edge_mem = sum([sys.getsizeof(e) for e in user_movie_graph.edges])
node_mem = sum([sys.getsizeof(n) for n in user_movie_graph.nodes])

print("Edge memory:", edge_mem / (1024**2),"MB")
print("Node memory:", node_mem / (1024**2),"MB")
print("Total memory:", (edge_mem + node_mem) / (1024**2), "MB")

Edge memory: 5.385009765625 MB
Node memory: 0.7169580459594727 MB
Total memory: 6.101967811584473 MB


In [11]:
users = {n for n, d in user_movie_graph.nodes(data=True) if d["bipartite"] == 0}
print(f"Users: {list(users)[:10]}")
print(f"Number of users: {len(users)}")

Users: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Number of users: 610


In [14]:
movies = {n for n, d in user_movie_graph.nodes(data=True) if d["bipartite"] == 1}
print(f"Movies: {list(movies)[:10]}")
print(f"Number of movies: {len(movies)}")

Movies: ['Just Jim (2015)', 'Opera (1987)', '00 Schneider - Jagd auf Nihil Baxter (1994)', 'Daredevil (2003)', 'Dreamgirls (2006)', 'Black Rain (1989)', 'Sapphire Blue (2014)', 'Howling, The (1980)', 'Hoosiers (a.k.a. Best Shot) (1986)', 'Psycho II (1983)']
Number of movies: 9719


In [13]:
print(nx.is_bipartite(user_movie_graph))
print(nx.is_connected(user_movie_graph))

True
True


In [15]:
# Project the graph using weights

start_time = time.time()
movie_movie_graph = nx.bipartite.weighted_projected_graph(user_movie_graph, movies)
print("--- %s seconds ---" % (time.time() - start_time))

--- 113.58181571960449 seconds ---


In [15]:
print(len(movie_movie_graph.nodes()))
print(f"Nodes in movie_movie_graph: {list(movie_movie_graph.nodes(data=True))[:10]}")

9719
Nodes in movie_movie_graph: [('Upstream Color (2013)', {'bipartite': 1, 'genre': 'Romance|Sci-Fi|Thriller', 'movieId': 99917}), ('Mansfield Park (1999)', {'bipartite': 1, 'genre': 'Comedy|Drama|Romance', 'movieId': 3079}), ('We Own the Night (2007)', {'bipartite': 1, 'genre': 'Crime|Drama', 'movieId': 55272}), ('Last Life in the Universe (Ruang rak noi nid mahasan) (2003)', {'bipartite': 1, 'genre': 'Drama|Romance', 'movieId': 27722}), ('Mr Hublot (2013)', {'bipartite': 1, 'genre': 'Animation|Comedy', 'movieId': 115819}), ('Lion (2016)', {'bipartite': 1, 'genre': 'Drama', 'movieId': 165551}), ('T2 3-D: Battle Across Time (1996)', {'bipartite': 1, 'genre': '(no genres listed)', 'movieId': 172497}), ('Higher Learning (1995)', {'bipartite': 1, 'genre': 'Drama', 'movieId': 358}), ('Baby Mama (2008)', {'bipartite': 1, 'genre': 'Comedy', 'movieId': 59258}), ('Beverly Hills Cop II (1987)', {'bipartite': 1, 'genre': 'Action|Comedy|Crime|Thriller', 'movieId': 4084})]


In [16]:
print(len(movie_movie_graph.edges()))
print(f"Edges in movie_movie_graph: {list(movie_movie_graph.edges(data=True))[:10]}")

13154589
Edges in movie_movie_graph: [('Upstream Color (2013)', 'Kiss of the Dragon (2001)', {'weight': 2}), ('Upstream Color (2013)', "National Lampoon's Van Wilder (2002)", {'weight': 2}), ('Upstream Color (2013)', 'Last Life in the Universe (Ruang rak noi nid mahasan) (2003)', {'weight': 2}), ('Upstream Color (2013)', 'Mondo Cane (1962)', {'weight': 1}), ('Upstream Color (2013)', 'Treasure Island (2012)', {'weight': 1}), ('Upstream Color (2013)', 'Mr Hublot (2013)', {'weight': 1}), ('Upstream Color (2013)', 'Next Karate Kid, The (1994)', {'weight': 1}), ('Upstream Color (2013)', 'River Runs Through It, A (1992)', {'weight': 1}), ('Upstream Color (2013)', "Big Momma's House (2000)", {'weight': 1}), ('Upstream Color (2013)', 'Transcendence (2014)', {'weight': 1})]


In [17]:
# Check memory consumption
edge_mem = sum([sys.getsizeof(e) for e in movie_movie_graph.edges])
node_mem = sum([sys.getsizeof(n) for n in movie_movie_graph.nodes])

print("Edge memory:", edge_mem / (1024**2),"MB")
print("Node memory:", node_mem / (1024**2),"MB")
print("Total memory:", (edge_mem + node_mem) / (1024**2), "MB")

Edge memory: 702.5308456420898 MB
Node memory: 0.7006692886352539 MB
Total memory: 703.2315149307251 MB


In [18]:
print(nx.is_connected(movie_movie_graph))

True


In [18]:
print([0])

('Just Jim (2015)', 'Toys (1992)', {'weight': 1})


In [ ]:
dct = {}
for e in list(list(movie_movie_graph.edges(data=True)).edges):
    dict[e[2]['weight']] += 1

In [20]:
#pip install neo4j

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Connect to Neo4j
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"
password = "testtest"

driver = GraphDatabase.driver(uri, auth=(username, password))

In [34]:
upload = False

In [32]:
# Delete all nodes and relationships from Neo4j
if upload:
    def delete_all(tx):
        tx.run("MATCH ()-[r]->() DELETE r")
        tx.run("MATCH (n) DELETE n")
            
    with driver.session() as session:
        session.execute_write(delete_all)

In [33]:
# Save the user_movie_graph in Neo4j

if upload:
    def create_nodes_and_relationships(tx, userId, movieId, rating, title, genres):
        tx.run("MERGE (u:User {userId: $userId}) "
            "MERGE (m:Movie {movieId: $movieId, title: $title}) "
            "MERGE (u)-[:RATED {rating: $rating}]->(m)",
            userId=userId, movieId=movieId, rating=rating, title=title)
    
    start_time = time.time()
    with driver.session() as session:
        for _, row in user_movie_matrix.iterrows():
            session.execute_write(create_nodes_and_relationships, row["userId"], row["movieId"], row["rating"], row["title"], row["genres"])
    print("--- %s seconds ---" % (time.time() - start_time))

--- 1773.1200823783875 seconds ---


In [ ]:
upload_movie_movie_graph = False

In [18]:
# Save the movie_movie_graph in Neo4j

if upload_movie_movie_graph:
    def create_movie_movie_relationships(tx, movie1, movie2, weight):
        tx.run("MATCH (m1:Movie {title: $movie1}) "
            "MATCH (m2:Movie {title: $movie2}) "
            "MERGE (m1)-[:SIMILAR {weight: $weight}]->(m2)",
            movie1=movie1, movie2=movie2, weight=weight)
        
    with driver.session() as session:
        for movie1, movie2, weight in movie_movie_graph.edges(data=True):
            session.execute_write(create_movie_movie_relationships, movie1, movie2, weight["weight"])

L'idea era caricare on demand il grafo dal database se questo ci poteva aiutare con le performances, ma non è più un approccio che stiamo seguendo

PageRank misura l'_importanza_ di una pagina web sulla base del numero di link entranti, mentre i link uscenti _distribuiscono_ quella stessa importanza alle pagine raggiunte.

Se il grafo è rappresentato tramite _matrice di adiacenza_, $L_{ij} = 1$ se esiste un link che permette di passare dalla pagina _j_ alla pagina _i_, $j \rightarrow i$, altrimenti $L_{ij} = 0$.
Se $m_{j} = \sum_{k=1}^{n} L_{kj}$ è il numero di pagine che vengono linkate da _j_, un possibile valore di PageRank, _BrokenRank_, per la pagina _i_ è:

$p_{i} = \sum_{j \rightarrow i} \frac{p_{j}}{m_{j}} = \sum_{j=1}^{n} \frac{L_{ij}}{m_{j}}p_{j}$

In notazione matriciale:

$p=\begin{bmatrix}
    p_{1} \\
    p_{2} \\
    \vdots \\
    p_{n} \\
\end{bmatrix},
L=\begin{bmatrix}
    L_{11} && L_{12} && \dots && L_{1n} \\
    L_{21} && L_{22} && \dots && L_{2n} \\
    \vdots && \vdots && \ddots && \vdots \\
    L_{n1} && L_{n2} && \dots && L_{nn} \\
\end{bmatrix},
M=\begin{bmatrix}
    m_{1} && 0 && \dots && 0 \\
    0 && m_{2} && \dots && 0 \\
    \vdots && \vdots && \ddots && \vdots \\
    0 && 0 && \dots && m_{n} \\
\end{bmatrix} \implies p = LM^{-1}p = Ap$ 

Perciò _p_ è un autovettore della matrice _A_ con autovalore 1. Sfruttando algoritmi noti è possibile trovare gli autovalori della matrice  _A_ e nel caso in cui questa sia __sparsa__ si ottengono anche prestazioni migliori, dato l'alto numero di valori nulli che si possono ignorare.

Utilizzando le catene di Markov, posso rappresentare il processo di navigazione di un utente. Ho _n_ stati che definiscono una matrice di transizione di stato _P_ di dimensione $n \times n$ dove $P_{ij} è la probabilità di andare dalla pagina _i_ alla pagina _j_.

Se $p^{(0)}$ è il vettore con le probabilità iniziali, si vede come $p^{(1)} = P^{T}p^{(0)}$ è il vettore con tutte le probabilità di trovarsi in un certo stato _i_ dopo un singolo step.

Sia $A^{T}$ la nostra matrice di transizione, con $(A^{T})_{ij} = L_{ij}/m_{i}$. La nostra catena di Markov diventa:

$P_{ij} = \begin{cases}
    \frac{1}{m_{i}} & \text{if } i \rightarrow j \\
    0 & \text{altrimenti}
\end{cases}$

Una _distribuzione stazionaria_ per la catena di Markov è un vettore di probabilità _p_ con $p = Ap$, ovvero dopo uno step la distribuzione rimane invariata.
Se la catena di Markov è _connessa_, ovvero ogni stato è raggiungibile da ogni altro, la distribuzione _p_ esiste ed è _unica_.

Il problema è che non possiamo soddisfare queste ipotesi con il nostro grafo in quanto:
- Non tutti gli utenti hanno visto tutti i film
- Anche nei grafi di proiezione non è detto che tra ogni coppia di film esista un valore di similarità definito.

Questo porterebbe a delle dei vettori _p_ ambigui.

Per questo PageRank è definito modificando BrokenRank:

$p_{i} = \frac{1-d}{n} + d(\sum_{j=1}^{n} \frac{L_{ij}}{m_{j}}p_{j})$, con $ 0 < d < 1$, che in notazione matriciale diventa:
$p = (\frac{1-d}{n}E + dLM^{-1})p$, s.t. $\sum_{i=1}^{n} p_{i} = 1$, con _E_ matrice $n \times n$ di soli 1

Sia $ A = \frac{1-d}{n}E + dLM^{-1}$ e si consideri la catena di Markov con matrice di transizione $A^{T}$ t.c. $(A^{T})_{ij} = (1 - d)/n + d L_{ij}/m_{i}$. La catena può essere descritta come:

$P_{ij} = \begin{cases}
    (1 - d)/n + d / m_{i} & \text{if } i \rightarrow j \\
    (1 - d)/n & \text{altrimenti}
\end{cases}$, dove $(1 -d)/n$ è la probabilità di saltare verso una pagina non linkata. Questo risolve i problemi di connettività ed è ancora applicabile al nostro caso, in quanto è accettabile pensare che un utente, solito a vedere una certa categoria di film, provi a vedere un film totalmente al di fuori dei suoi standard.

Sfruttando le proprietà della catena di markov definita inoltre, è possibile evitare i classici algoritmi di eigendecomposition di complessità $O(n^{3})$ utilizzando una qualsiasi distribuzione iniziale $p^{(0)}$ e calcolando
- $p^{(1)} = Ap^{(0)}$
- $p^{(2)} = Ap^{(1)}$
- $\vdots$
- $p^{(t)} = Ap^{(t-1)}$

Quando $t \rightarrow \inf$, $p^{(t)} \rightarrow p$, perciò con un numero di iterazioni sufficientemente grande _t_ si ottiene una complessità $O(tn^{2}) = O(n^{2})$ per n grandi.

L'ultimo cambiamento introdotto è una generalizzazione della matrice A:

$A^{T} = d(L^{T}D_{L}^{-1} + kg_{L}^{T}) + (1-d)ke^{T}$, dove L è la matrice di adiacenza, $D_{L}$ è la matrice diagonale contenente il grado uscente di ogni nodo, mentre $g_{L}^{T}$ contiene gli indici delle pagine che non hanno link uscenti.

_k_ è il vettore di __personalizzazione__. Se tende ad una distribuzione uniforme fornisce un risultato simile al PageRank classico, mentre più il vettore è polarizzato, più l'algoritmo è portato a teleportarsi verso i nodi specificati.

Nel nostro caso il vettore di personalizzazione dovrà tenere conto dei film visti dall'utente per dare più importanza a quel vicinato di film e polarizzare PageRank nel dare punteggi più alti a film simili a quelli già visti.

In [9]:
# 0: User, 1: Movie
def filter_nodes(graph: nx.Graph, node_type: int):
    return [n for n, d in graph.nodes(data=True) if d["bipartite"] == node_type]

In [10]:
def create_preference_vector_debug(user_id: int, user_movie_graph: nx.Graph):
    edges = {m: v for _, m, v in user_movie_graph.edges(user_id, data="weight")}
    
    print(f"Edges for user {user_id}: {list(edges)[:10]}")
    print(f"Number of edges for user {user_id}: {len(edges)}")
        
    for k, v in edges.items():
        print(k,v)

    tot = sum(edges.values())
    
    print(f"Total for user {user_id}: {tot}")

    if tot > 0:
        return len(edges), {
            movie: edges.get(movie, 0) / tot # get(key, default) : se la chiave non esiste ritorna il default
            for movie in filter_nodes(user_movie_graph, 1) # 1 : Movie
        }
    else:
        return len(edges), {
            # movie: 1 for movie in filter_nodes(user_movie_graph, 1) # <- Penso che dovremmo metterci zero come peso in questo caso
            movie: 1/len(movie) for movie in filter_nodes(user_movie_graph, 1) # 
        }

Il vettore di personalizzazione assegna 0 a tutti i film non visti, mentre assegna valori normalizzati in base alla rating a tutti gli altri archi. Per come è stato costruito il vettore dei pesi delle ratings, al momento voti come 2, 2.5 e 3 vengono considerati allo stesso modo di film non visti

In [23]:
# Test the function
debug = True
num, p_vec = create_preference_vector_debug(1, user_movie_graph)

Edges for user 1: ['Toy Story (1995)', 'Grumpier Old Men (1995)', 'Heat (1995)', 'Seven (a.k.a. Se7en) (1995)', 'Usual Suspects, The (1995)', 'From Dusk Till Dawn (1996)', 'Bottle Rocket (1996)', 'Braveheart (1995)', 'Rob Roy (1995)', 'Canadian Bacon (1995)']
Number of edges for user 1: 232
Toy Story (1995) 1
Grumpier Old Men (1995) 1
Heat (1995) 1
Seven (a.k.a. Se7en) (1995) 1.2
Usual Suspects, The (1995) 1.2
From Dusk Till Dawn (1996) 0.1
Bottle Rocket (1996) 1.2
Braveheart (1995) 1
Rob Roy (1995) 1.2
Canadian Bacon (1995) 1.2
Desperado (1995) 1.2
Billy Madison (1995) 1.2
Clerks (1994) 0.1
Dumb & Dumber (Dumb and Dumber) (1994) 1.2
Ed Wood (1994) 1
Star Wars: Episode IV - A New Hope (1977) 1.2
Pulp Fiction (1994) 0.1
Stargate (1994) 0.1
Tommy Boy (1995) 1.2
Clear and Present Danger (1994) 1
Forrest Gump (1994) 1
Jungle Book, The (1994) 1.2
Mask, The (1994) 1
Blown Away (1994) 0.1
Dazed and Confused (1993) 1
Fugitive, The (1993) 1.2
Jurassic Park (1993) 1
Mrs. Doubtfire (1993) 0.1
Sch

In [24]:
print(f"Number of movies rated by the user: {num}")
weight_dict = {}
for k, v in p_vec.items():
    if v in weight_dict:
        weight_dict[v] += 1
    else:
        weight_dict[v] = 1

print(weight_dict)

total = 0
for k, v in weight_dict.items():
    if k != 0.0:
        total += v

print("Number of values different from zero", total)
if total != num:
    print("Alcuni dei film visti dall'utente hanno peso 0")   

Number of movies rated by the user: 232
{0.004468275245755149: 76, 0.0053619302949061785: 124, 0.0004468275245755149: 26, -0.0022341376228775744: 5, -0.004915102770330664: 1, 0.0: 9487}
Number of values different from zero 232


In [15]:
def predict_user_debug(user_id, user_movie_graph: nx.Graph, movie_movie_graph: nx.Graph):
    _, p_vec = create_preference_vector_debug(user_id, user_movie_graph)
    
    print(f"Preference vector for user {user_id}: {list(p_vec)[:10]}")  

    already_seen = [m for _, m, v in user_movie_graph.edges(user_id, data="weight")]

    print(f"Already seen movies for user {user_id}: {list(already_seen)[:10]}")  

    if len(already_seen) < 1:
        return []
    
    item_rank = nx.pagerank(movie_movie_graph, personalization=p_vec, alpha=0.85, weight="weight")
    
    print(f"Item rank for user {user_id}: {list(item_rank)[:10]}") 

    s_t = [
        x for x in sorted(
            movie_movie_graph.nodes(), key=lambda x: item_rank[x] if x in item_rank else 0, reverse=True
            )
        if x not in already_seen
        ]
    
    return s_t[:10]

In [ ]:
#pip install scipy

In [16]:
# Test the prediction
user = 10
start_time = time.time()
s_t = predict_user_debug(user, user_movie_graph, movie_movie_graph)
print("--- %s seconds ---" % (time.time() - start_time))

for movie in s_t:
    print(movie)

Edges for user 10: ['Pulp Fiction (1994)', 'Forrest Gump (1994)', 'Aladdin (1992)', 'Pretty Woman (1990)', 'Casablanca (1942)', 'Mary Poppins (1964)', 'Dirty Dancing (1987)', 'Graduate, The (1967)', 'When Harry Met Sally... (1989)', 'As Good as It Gets (1997)']
Number of edges for user 10: 140
Pulp Fiction (1994) -1.1
Forrest Gump (1994) 0.5
Aladdin (1992) 1
Pretty Woman (1990) 0.5
Casablanca (1942) 1
Mary Poppins (1964) -1.2
Dirty Dancing (1987) 0.1
Graduate, The (1967) 0.1
When Harry Met Sally... (1989) 0.1
As Good as It Gets (1997) 0.5
Mulan (1998) 1
Matrix, The (1999) -1.2
Notting Hill (1999) 0.5
Sixth Sense, The (1999) -1.2
American Beauty (1999) -1.1
Fight Club (1999) -1.2
Gladiator (2000) 1
Bring It On (2000) 0.1
Bridget Jones's Diary (2001) 0.5
Shrek (2001) 1.1
Legally Blonde (2001) 1.1
Lord of the Rings: The Fellowship of the Ring, The (2001) 1
Beautiful Mind, A (2001) 1
Walk to Remember, A (2002) 0.1
About a Boy (2002) 0.5
Sweet Home Alabama (2002) 0.1
Maid in Manhattan (2002

In [35]:
upload_test = False

In [36]:
# Delete old test
if upload_test:
    def delete_test(tx, userId, movieId):
        tx.run("MATCH (u:User {userId: $userId})-[r:RECOMMENDED]->(m:Movie {movieId: $movieId}) DELETE r", userId=userId, movieId=movieId)

    with driver.session() as session:
        for movieId in s_t:           
            session.execute_write(delete_test, user, movieId)

In [37]:
# Upload the test
if upload_test:
    def create_recommendations(tx, userId, recs):
            for rec in recs:
                tx.run("MATCH (u:User {userId: $userId}), (m:Movie {title: $title})"
                    "MERGE (u)-[:RECOMMENDED]->(m)",
                    userId=userId, title=rec)

    with driver.session() as session:
        session.execute_write(create_recommendations, user, s_t)

In [14]:
def create_preference_vector(user_id: int, user_movie_graph: nx.Graph):
    edges = {m: v for _, m, v in user_movie_graph.edges(user_id, data="weight")}
    tot = sum(edges.values())
    if tot > 0:
        return {
            movie: edges.get(movie, 0) / tot
            for movie in filter_nodes(user_movie_graph, 1)
        }
    else:
        return {
            movie: 1/len(movie) for movie in filter_nodes(user_movie_graph, 1) 
        }
        
def predict_user(user_id, user_movie_graph: nx.Graph, movie_movie_graph: nx.Graph):
    p_vec = create_preference_vector(user_id, user_movie_graph)
    already_seen = [m for _, m, v in user_movie_graph.edges(user_id, data="weight")]
    if len(already_seen) < 1:
        return []
    item_rank = nx.pagerank(movie_movie_graph, personalization=p_vec, alpha=0.85, weight="weight")
    s_t = [
        x for x in sorted(
            movie_movie_graph.nodes(), key=lambda x: item_rank[x] if x in item_rank else 0, reverse=True
            )
        if x not in already_seen
        ]
    
    return s_t[:10]

In [10]:
upload_predictions = True

In [ ]:
def create_recommendations(tx, userId, recs):
    for rec in recs:
        tx.run("MATCH (u:User {userId: $userId}), (m:Movie {title: $title})"
            "MERGE (u)-[:RECOMMENDED]->(m)",
            userId=userId, title=rec)
            
if upload_predictions:
    with driver.session() as session:
        start_time = time.time()
        for user in filter_nodes(user_movie_graph, 0):
            recs = predict_user(user, user_movie_graph, movie_movie_graph)    
            session.execute_write(create_recommendations, user, recs)
        print("--- %s seconds ---" % (time.time() - start_time))
    

In [17]:
driver.close()

In [ ]:
exit()